# 1. Import data

The primary dataset includes the selling retail data of 44 products for a hundred weeks. There are 8 recorded features in the initial dataset: week, product item, weekly_sales, functionality, featured on the web page, color, and vendor. After some pre-processing steps, we have our dataset ready for interpretation and forecasting. Please visit the  following book to be more familiar with the features and pre-processing steps:

https://www.springerprofessional.de/en/demand-prediction-in-retail/19981986.


In [40]:
import pandas as pd
sales=pd.read_csv("data_processed.csv")
sales

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,...,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
0,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,2018-08-27,44,20.0,53.99,42.38,43.99,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
4308,2018-09-03,44,14.0,52.99,53.99,42.38,0,2,0,0,...,0,0,0,0,0,1,0,0,0,0
4309,2018-09-10,44,22.0,44.99,52.99,53.99,1,2,0,0,...,0,0,0,0,0,1,0,0,0,0
4310,2018-09-17,44,28.0,42.99,44.99,52.99,1,2,0,0,...,0,0,0,0,0,1,0,0,0,0


# 2. Centralized and decentralized approaches

There are two general approaches to predict the demand of products. The first is to consider a single predictor for all product items, called centralized method, and the other approach is to consider products separately, called decentralized method. With regard to prediction accuracy, I found that there is no rigid priority to use either centralized or decentralized appraoches. For the prediction, we use the linear regression method. The 'res' dataframe is created to save the prediction accuracy of methods.


In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
from sklearn.metrics import r2_score, mean_squared_error

res=pd.DataFrame(index=['R2'])

## 2.1. Structuring the dataset

Before proceeding to the approaches mentioned above, we need to split our data into train and test datasets for each specific product.

In [42]:
skuSet = list(sales.sku.unique())
skuData = {}
colnames = [i for i in sales.columns if i not in ["week","weekly_sales","sku"]]
for i in skuSet:
  df_i = sales[sales.sku == i]
  skuData[i] = {'X': df_i[colnames].values,
                'y': df_i.weekly_sales.values}
    
X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 

## 2.2. Centralized method

 Once the train and test datasets are created for each product, we combine them to deploy our centralized solution method and evaluate it.

In [43]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

model_cen = LinearRegression().fit(X_cen_train, y_train)

print('Centralized method with linear regression R2:',
      round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('Centralized method with linear regression MSE:',
      round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

res['Centralized(LR)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Centralized method with linear regression R2: 0.114
Centralized method with linear regression MSE: 98086.301


## 2.3. Decentralized mehod

In this subsection, a dictionary of prediction mehods is created for each product and the total accuracy of it is caculated then.


In [44]:
y_pred = []
skuModels = {}

for i in skuSet:
 #one model for each item, fitted on training set
 model_i = OLS(y_dict[i]['train'], X_dict[i]['train'])
 skuModels[i] = model_i.fit()

 #compute and concatenate prediction of the model i on item i
 y_pred += list(skuModels[i].predict(X_dict[i]['test']))


#computing overall performance metrics on y_pred and y_test:
print('Decentralized method with linear regression R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Decentralized method with linear regression MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(LR)']=[r2_score(y_test, np.array(y_pred))]

Decentralized method with linear regression R2: 0.517
Decentralized method with linear regression MSE: 53537.475


# 3. Prediction with aggregated seasonality

A common approach in retail is to covert the existing variable (or columns) into multiple variables, based on the avaialable product items. In this section, all the features, except for the seasonality, are cosidered at product item level inside the dataset. This method is called 'feature-fixed effect'. Note that this method may result in a better prediction for the centralized method, but the decentraized method already considers the dataset at the product item levels. As a result, the prediction accuracy of the decentralized method may not improve.

## 3.1. Structuring the dataset

In [45]:
sales_fe_sku = sales.copy()
sales_fe_sku = pd.get_dummies(data=sales_fe_sku, columns=['sku'])
sales_fe_sku["sku"] = sales["sku"] 


colnames_to_fix = [i for i in sales.columns if i not in ["week","weekly_sales","sku",
                                                         'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
                                                         'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 
                                                         'month_12']]

sales_seasonality = sales_fe_sku.copy()

for feature in colnames_to_fix:
  for i in range(1,45):
    sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]

skuSet = list(sales.sku.unique()) #the SKU numbers do not change
skuData = {}
colnames = [i for i in sales_seasonality.columns if i not in ["week","weekly_sales","sku"] and i not in colnames_to_fix]
for i in skuSet:
  df_i = sales_seasonality[sales_seasonality.sku == i]
  skuData[i] = {'X': df_i[colnames].values,
                'y': df_i.weekly_sales.values}
    


X_dict = {}
y_dict = {}

y_test = []
y_train = []

for i in skuSet:
  
  X_train_i,X_test_i = train_test_split(skuData[i]["X"], shuffle=False, train_size=0.7) #split for X
  y_train_i,y_test_i = train_test_split(skuData[i]["y"], shuffle=False, train_size=0.7) #split for y 

  X_dict[i] = {'train': X_train_i, 'test': X_test_i} #filling dictionary
  y_dict[i] = {'train': y_train_i, 'test': y_test_i}

  y_test += list(y_test_i) 
  y_train += list(y_train_i) 



C:\Users\NOOR~1\AppData\Local\Temp/ipykernel_14140/3443539507.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  sales_seasonality[str(feature)+"_fixed_effect_"+str(i)] = sales_seasonality[feature]*sales_seasonality["sku_"+str(i)]


## 3.2. Centralized method

In [46]:
X_cen_train = X_dict[skuSet[0]]['train'] #initialization with item 0
X_cen_test = X_dict[skuSet[0]]['test']

for i in skuSet[1:]: #Iteration over items
    X_cen_train = np.concatenate((X_cen_train, X_dict[i]['train']), axis = 0) #Bringing together the training set
    X_cen_test = np.concatenate((X_cen_test, X_dict[i]['test']), axis = 0)

model_cen = LinearRegression(fit_intercept=True).fit(X_cen_train, y_train)
print('Seasonality aggregated (LR) R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('Seasonality aggregated (LR) MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

res['Centralized(SA-LR)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Seasonality aggregated (LR) R2: 0.616
Seasonality aggregated (LR) MSE: 42516.024


## 3.3. Decentralized method

In [47]:
y_pred = []
skuModelsElastic = {}

for i in skuSet:
    skuModels[i] = LinearRegression(fit_intercept=True).fit(X_dict[i]["train"],y_dict[i]["train"])
    y_pred += list(skuModels[i].predict(X_dict[i]['test']))

#computing overall performance metrics on y_pred and y_test:
print('OOS R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('OOS MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(SA-LR)']=[r2_score(y_test, np.array(y_pred))]

OOS R2: 0.517
OOS MSE: 53537.475


# 4. Regularization

As you noticed, the dataset in the previous section includes so many variables. Therefore, some problems like overfitting or multicolinearity may occur, reducing the prediction accuracy or decrasing the reliablity on coefficients. In response, we deploy the Elasticnet method, which has the advantage of both Lasso and Ridge regressions. Note that a hypeparameter tuning algorithm is used to determine the best value of parameters of the Elasticnet method.

In [48]:
from sklearn.linear_model import ElasticNet

## 4.1. Centralized method

In this subsection, we consider the seasonality aggregated approach (Section 3.2) for the centralized prediction method and use Elasticnet for prediction.


In [ ]:
'''
# Hyperparameter tuning
BestR2 = -1
BestPar = [-1,-1]

idx1 = [0.1*i for i in range(1,11)] # Set of values for alpha
idx2 = [0.1*i for i in range(11)]   # Set of values for l1_ratio

#hyperparameter tuning for centralized
for i in idx1:
    for j in idx2:
        model_cen = ElasticNet(alpha= i,l1_ratio=j)
        model_cen.fit(X_cen_train, y_train)
        R2 = r2_score(y_test, model_cen.predict(X_cen_test))
        if R2>BestR2:
            BestR2 = R2
            BestPar[0] = i
            BestPar[1] = j
print('The best value of alpha:',BestPar[0])
print('The best value of l1_ratio:',BestPar[1])
'''

In [49]:
#model_cen = ElasticNet(alpha=BestPar[0],l1_ratio=BestPar[1])
model_cen = ElasticNet(alpha=0.1,l1_ratio=1)
model_cen.fit(X_cen_train, y_train)
print('Seasonality aggregated (EN) R2:', round(r2_score(y_test, model_cen.predict(X_cen_test)),3))  
print('Seasonality aggregated (EN) MSE:', round(mean_squared_error(y_test, model_cen.predict(X_cen_test)),3))

res['Centralized(SA-EN)']=[r2_score(y_test, model_cen.predict(X_cen_test))]

Seasonality aggregated (EN) R2: 0.632
Seasonality aggregated (EN) MSE: 40766.049


E:\Sotwares\Anaconda\Installation\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10229889.879478272, tolerance: 21709.014304244654
  model = cd_fast.enet_coordinate_descent(


## 4.2. Decentralized method

In this subsection, we consider the seasonality aggregated approach (Section 3.3) for the centralized prediction method and use Elasticnet for prediction.

In [ ]:
'''
#Hyperparameter Tuning
BestR2 = -1
BestPar = [-1,-1]

idx1 = [0.1*i for i in range(1,11)] # Set of values for alpha
idx2 = [0.1*i for i in range(11)]   # Set of values for l1_ratio

for i in idx1:
    for j in idx2:
        y_pred = []
        y_test = []
        for k in skuSet:
            elastic = ElasticNet(alpha= i,l1_ratio=j)
            ModelsElastic = elastic.fit(X_dict[k]["train"],y_dict[k]["train"])
            y_pred += list(ModelsElastic.predict(X_dict[k]['test']))
            y_test += list(y_dict[k]["test"])
        R2 = r2_score(np.array(y_test), np.array(y_pred))
        if R2>BestR2:
            BestR2 = R2
            BestPar[0] = i
            BestPar[1] = j
            
print('The best value of alpha:',BestPar[0])
print('The best value of l1_ratio:',BestPar[1])
'''

In [51]:
y_pred = []
skuModels = {}

for i in skuSet:
#    elastic = ElasticNet(alpha=BestPar[0],l1_ratio=BestPar[1])
    elastic = ElasticNet(alpha=1,l1_ratio=0.9)
    skuModels[i] = elastic.fit(X_dict[i]["train"],y_dict[i]["train"])
    y_pred += list(skuModels[i].predict(X_dict[i]['test']))

#computing overall performance metrics on y_pred and y_test:
print('Seasonality aggregated (EN) R2:',round(r2_score(y_test, np.array(y_pred)),3))
print('Seasonality aggregated (EN) MSE:', round(mean_squared_error(y_test, np.array(y_pred)),3))

res['Decentralized(SA-EN)']=[r2_score(y_test, np.array(y_pred))]

Seasonality aggregated (EN) R2: 0.595
Seasonality aggregated (EN) MSE: 44882.604


# 5. Results

In [52]:
res

,Centralized(LR),Decentralized(LR),Centralized(SA-LR),Decentralized(SA-LR),Centralized(SA-EN),Decentralized(SA-EN)
R2,0.114249,0.516539,0.616066,0.516539,0.631869,0.594695


From the above chart, we notice that the most accurate approach is to make seasonality aggregated data and use Elasticnet ia a centralized approach. 
It is evident that the seasonality aggregated data plays an important role in the prediction accuracy of the centralized method. 